In [ ]:
#bibliotecas necessárias 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

## ARIMA

In [ ]:
# Decomposição e dessazonalização
ts = df['preco_soja_atualizado']
decomposition = seasonal_decompose(ts, model='multiplicative', period=12)
seasonal = decomposition.seasonal
ts_deseasonalized = ts / seasonal

# Ajuste ARIMA
model_arima = ARIMA(ts_deseasonalized, order=(2, 1, 2))
model_arima_fit = model_arima.fit()

# Separar treino/teste (80/20)
split = int(len(ts_deseasonalized) * 0.80)
test_dates = ts_deseasonalized.index[split+1:]
forecast_arima = model_arima_fit.forecast(steps=len(ts) - split - 1)

# Reaplicar sazonalidade
seasonal_avg = seasonal.groupby(seasonal.index.month).mean()
forecast_arima_real = forecast_arima.values * [seasonal_avg[d.month] for d in test_dates]
real = ts[test_dates]

## LSTM

In [ ]:
# Série diferenciada e dessazonalizada
ts_lstm = ts_deseasonalized.diff().dropna()
scaler = MinMaxScaler()
ts_scaled = scaler.fit_transform(ts_lstm.values.reshape(-1, 1))

# Criar janelas
def create_sequences(data, window):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(data[i])
    return np.array(X), np.array(y)

window_size = 24
X, y = create_sequences(ts_scaled, window_size)
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
# Treino e teste
X_train, X_test = X[:split-window_size], X[split-window_size:]
y_train, y_test = y[:split-window_size], y[split-window_size:]
test_dates_lstm = ts_deseasonalized.index[split + window_size:]

In [ ]:
# === LSTM com mais camadas ===
model = Sequential()
model.add(LSTM(200, return_sequences=True, activation='relu', input_shape=(window_size, 1)))
model.add(Dropout(0.2))

## Segunda camada LSTM (mais profunda)
model.add(LSTM(100, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))

## Segunda camada LSTM (mais profunda)
model.add(LSTM(50, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))

## terceira camada LSTM (mais profunda)
model.add(LSTM(25, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.005), loss='mse')


## Primeira camada LSTM (retorna sequências para empilhamento)
#model.add(LSTM(100, return_sequences=True, activation='relu', input_shape=(window_size, 1)))
#model.add(Dropout(0.2))

## Segunda camada LSTM (mais profunda)
#model.add(LSTM(50, return_sequences=False, activation='relu'))
#model.add(Dropout(0.2))

#model.compile(optimizer=Adam(learning_rate=0.05), loss='mse')

# treinar
model.fit(X_train, y_train, epochs=500, verbose=0)

# Previsão
pred_lstm = model.predict(X_test)
pred_lstm_inv = scaler.inverse_transform(pred_lstm)

# Reintegrar os valores previstos com a série base
start_value = ts_deseasonalized.iloc[split + window_size - 1]
pred_lstm_deseasonalized = np.r_[start_value, start_value + np.cumsum(pred_lstm_inv.flatten())]

# Aplicar sazonalidade novamente com segurança
seasonality_lstm = [seasonal_avg.get(d.month, 1.0) for d in test_dates_lstm]
n = min(len(pred_lstm_deseasonalized[1:]), len(test_dates_lstm))
forecast_lstm_real = pred_lstm_deseasonalized[1:n+1] * [seasonal_avg.get(d.month, 1.0) for d in test_dates_lstm[:n]]
real_lstm = ts[test_dates_lstm[:n]]

## Avaliação

In [ ]:
# Garantir que os índices são datetime
if isinstance(test_dates, pd.PeriodIndex):
    test_dates = test_dates.to_timestamp()

if isinstance(test_dates_lstm, pd.PeriodIndex):
    test_dates_lstm = test_dates_lstm.to_timestamp()

# ARIMA - Avaliação direta
rmse_arima = np.sqrt(mean_squared_error(real, forecast_arima_real))
mape_arima = mean_absolute_percentage_error(real, forecast_arima_real)

# LSTM - Alinhar índice com série real
n = min(len(pred_lstm_deseasonalized[1:]), len(test_dates_lstm))
forecast_lstm_real = pred_lstm_deseasonalized[1:n+1] * [seasonal_avg.get(d.month, 1.0) for d in test_dates_lstm[:n]]

# Garantir real_lstm somente com datas válidas
real_lstm = pd.Series(
    data=[ts.get(d, np.nan) for d in test_dates_lstm[:n]],
    index=pd.to_datetime(test_dates_lstm[:n])
).dropna()

# Alinhar previsão ao índice real
forecast_lstm_real = pd.Series(
    data=forecast_lstm_real[:len(real_lstm)],
    index=real_lstm.index
)

# LSTM - Avaliação
rmse_lstm = np.sqrt(mean_squared_error(real_lstm, forecast_lstm_real))
mape_lstm = mean_absolute_percentage_error(real_lstm, forecast_lstm_real)

# Resultados
print(f"ARIMA - RMSE: {rmse_arima:.4f} | MAPE: {mape_arima:.2%}")
print(f"LSTM  - RMSE: {rmse_lstm:.4f} | MAPE: {mape_lstm:.2%}")

# === Gráfico Final ===
plt.figure(figsize=(14,6))
plt.plot(real.index, real.values, label='Real (ARIMA)', linewidth=2)
plt.plot(real.index, forecast_arima_real, label='ARIMA', linestyle='--')
plt.plot(real_lstm.index, forecast_lstm_real, label='LSTM', linestyle=':')
plt.title("Comparação de Previsões - ARIMA vs LSTM (Preço do Leite)")
plt.xlabel("Data")
plt.ylabel("Preço (R$)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()